In [1]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect
import sqlite3

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder


In [2]:
engine = create_engine("sqlite:///Resources/flights.db")

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['flights']

In [4]:
session = Session(engine)

In [5]:
columns = inspector.get_columns('flights')
for column in columns:
    print(column)

{'name': 'MONTH', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'DATE', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'DAY_OF_WEEK', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'AIRLINE', 'type': TEXT(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'FLIGHT_NUMBER', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'TAIL_NUMBER', 'type': TEXT(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'ORIGIN_AIRPORT', 'type': TEXT(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'DESTINATION_AIRPORT', 'type': TEXT(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'DISTANCE_miles', 'type': INTEGE

In [6]:
# Get the column names for the table
column_names = inspector.get_columns('flights')
for column_name in column_names:
    print(column_name["name"])

MONTH
DATE
DAY_OF_WEEK
AIRLINE
FLIGHT_NUMBER
TAIL_NUMBER
ORIGIN_AIRPORT
DESTINATION_AIRPORT
DISTANCE_miles
MINUTES_DELAY


In [ ]:
# Query All Records in the the Database
data = engine.execute("SELECT * FROM flights")

for record in data:
    print(record)

In [7]:
# Make a connection to the SQL database
conn = engine.connect()

In [9]:
# Query All Records in the the Database
flights_data_df = pd.read_sql("SELECT * FROM flights", conn)
flights_data_df.head()

,MONTH,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE_miles,MINUTES_DELAY
0,1,1,4,AS,98,N407AS,ANC,SEA,1448,-22
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,2330,-9
2,1,1,4,US,840,N171US,SFO,CLT,2296,5
3,1,1,4,AA,258,N3HYAA,LAX,MIA,2342,-9
4,1,1,4,AS,135,N527AS,SEA,ANC,1448,-21


In [10]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
flights_data_df = flights_data_df.drop(columns=['TAIL_NUMBER'], axis=1)
flights_data_df.head()

,MONTH,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE_miles,MINUTES_DELAY
0,1,1,4,AS,98,ANC,SEA,1448,-22
1,1,1,4,AA,2336,LAX,PBI,2330,-9
2,1,1,4,US,840,SFO,CLT,2296,5
3,1,1,4,AA,258,LAX,MIA,2342,-9
4,1,1,4,AS,135,SEA,ANC,1448,-21


In [ ]:
#the condition we’re testing for, the value to assign to our new column if that condition is <= 0 flight is early, and the value to assign if it is early is '0'
#if flight is delayed by 1 minute or more than assign it '1'
#flights_data_df['IS_Delayed'] = np.where(flights_data_df['MINUTES_DELAY']<= 0, '0', '1')
#flights_data_df.head(20)

In [11]:
flights_data_df['AIRLINE'].value_counts().head(20)

WN    221586
DL    147486
EV    111206
OO    107099
AA     97549
UA     87606
US     73942
MQ     65513
B6     48157
AS     29614
NK     19612
F9     14669
HA     14133
VX     10403
Name: AIRLINE, dtype: int64

In [12]:
flights_data_df['ORIGIN_AIRPORT'].value_counts().head(10)

ATL    66599
ORD    52961
DFW    50933
LAX    38473
DEN    38254
IAH    29802
PHX    29262
SFO    28428
LAS    25806
MCO    22575
Name: ORIGIN_AIRPORT, dtype: int64

In [13]:
flights_data_df.nunique()

MONTH                     3
DATE                     31
DAY_OF_WEEK               7
AIRLINE                  14
FLIGHT_NUMBER          6522
ORIGIN_AIRPORT          315
DESTINATION_AIRPORT     315
DISTANCE_miles         1271
MINUTES_DELAY           883
dtype: int64

In [14]:
flights_data_encoded = pd.get_dummies(flights_data_df, columns=["AIRLINE"])
flights_data_encoded.head()

,MONTH,DATE,DAY_OF_WEEK,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE_miles,MINUTES_DELAY,AIRLINE_AA,AIRLINE_AS,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
0,1,1,4,98,ANC,SEA,1448,-22,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,4,2336,LAX,PBI,2330,-9,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,4,840,SFO,CLT,2296,5,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1,4,258,LAX,MIA,2342,-9,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,4,135,SEA,ANC,1448,-21,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
OA_counts = flights_data_df.ORIGIN_AIRPORT.value_counts()
OA_counts

ATL    66599
ORD    52961
DFW    50933
LAX    38473
DEN    38254
       ...  
UST       33
BGR       22
PPG       21
ADK       20
ITH        6
Name: ORIGIN_AIRPORT, Length: 315, dtype: int64

In [16]:
# Determine which values to replace if counts are less than ...?
replace_OA = list(OA_counts[OA_counts <22574].index)

# Replace in dataframe
for app in replace_OA:
    flights_data_df.ORIGIN_AIRPORT = flights_data_df.ORIGIN_AIRPORT.replace(app,"Other")
    
# Check to make sure binning was successful
flights_data_df.ORIGIN_AIRPORT.value_counts()

Other    665482
ATL       66599
ORD       52961
DFW       50933
LAX       38473
DEN       38254
IAH       29802
PHX       29262
SFO       28428
LAS       25806
MCO       22575
Name: ORIGIN_AIRPORT, dtype: int64

In [17]:
DA_counts = flights_data_df.DESTINATION_AIRPORT.value_counts()
DA_counts.head(10)

ATL    66741
ORD    53060
DFW    51037
LAX    38463
DEN    38300
IAH    29820
PHX    29250
SFO    28437
LAS    25804
MCO    22586
Name: DESTINATION_AIRPORT, dtype: int64

In [18]:
# Determine which values to replace if counts are less than ...?
replace_DA = list(DA_counts[DA_counts <22584].index)

# Replace in dataframe
for app in replace_DA:
    flights_data_df.DESTINATION_AIRPORT = flights_data_df.DESTINATION_AIRPORT.replace(app,"Other")
    
# Check to make sure binning was successful
flights_data_df.DESTINATION_AIRPORT.value_counts()

Other    665077
ATL       66741
ORD       53060
DFW       51037
LAX       38463
DEN       38300
IAH       29820
PHX       29250
SFO       28437
LAS       25804
MCO       22586
Name: DESTINATION_AIRPORT, dtype: int64

In [19]:
# Generate our categorical variable lists
flight_cat = flights_data_df.dtypes[flights_data_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
flights_data_df[flight_cat].nunique()

AIRLINE                14
ORIGIN_AIRPORT         11
DESTINATION_AIRPORT    11
dtype: int64

In [20]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(flights_data_df[flight_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(flight_cat)
encode_df.head()

,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,...,DESTINATION_AIRPORT_DEN,DESTINATION_AIRPORT_DFW,DESTINATION_AIRPORT_IAH,DESTINATION_AIRPORT_LAS,DESTINATION_AIRPORT_LAX,DESTINATION_AIRPORT_MCO,DESTINATION_AIRPORT_ORD,DESTINATION_AIRPORT_Other,DESTINATION_AIRPORT_PHX,DESTINATION_AIRPORT_SFO
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# Merge one-hot encoded features and drop the originals
flights_data_df2 = flights_data_df.merge(encode_df,left_index=True,right_index=True)
flights_data_df2 = flights_data_df2.drop(flight_cat,1)
flights_data_df2.head()

,MONTH,DATE,DAY_OF_WEEK,FLIGHT_NUMBER,DISTANCE_miles,MINUTES_DELAY,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,...,DESTINATION_AIRPORT_DEN,DESTINATION_AIRPORT_DFW,DESTINATION_AIRPORT_IAH,DESTINATION_AIRPORT_LAS,DESTINATION_AIRPORT_LAX,DESTINATION_AIRPORT_MCO,DESTINATION_AIRPORT_ORD,DESTINATION_AIRPORT_Other,DESTINATION_AIRPORT_PHX,DESTINATION_AIRPORT_SFO
0,1,1,4,98,1448,-22,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,1,4,2336,2330,-9,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,1,4,840,2296,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,1,4,258,2342,-9,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,1,4,135,1448,-21,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
#the condition we’re testing for, the value to assign to our new column if that condition is <= 0 flight is early, and the value to assign if it is early is '0'
#if flight is delayed by 1 minute or more than assign it '1'
flights_data_df2['IS_Delayed'] = np.where(flights_data_df2['MINUTES_DELAY']<= 0, '0', '1')
flights_data_df2.head(20)

,MONTH,DATE,DAY_OF_WEEK,FLIGHT_NUMBER,DISTANCE_miles,MINUTES_DELAY,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,...,DESTINATION_AIRPORT_DFW,DESTINATION_AIRPORT_IAH,DESTINATION_AIRPORT_LAS,DESTINATION_AIRPORT_LAX,DESTINATION_AIRPORT_MCO,DESTINATION_AIRPORT_ORD,DESTINATION_AIRPORT_Other,DESTINATION_AIRPORT_PHX,DESTINATION_AIRPORT_SFO,IS_Delayed
0,1,1,4,98,1448,-22,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,1,1,4,2336,2330,-9,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,1,1,4,840,2296,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,1,1,4,258,2342,-9,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,1,1,4,135,1448,-21,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
5,1,1,4,806,1589,8,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
6,1,1,4,612,1299,-17,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
7,1,1,4,2013,2125,-10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
8,1,1,4,1112,1464,-13,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,1,1,4,1173,1747,-15,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [37]:
def preprocessing_inputs(new_flights_df):
    new_flights_df = new_flights_df.copy()
    
    #one-hot encoder nominal feature columns
    new_flights_df = onehot_encode(
        new_flights_df,
        column_dict={
            'AIRLINE': 'AL',
            'ORIGIN_AIRPORT': 'OA',
            'DESTINATION_AIRPORT': 'DA'
        })
    
    #fill remaining missing values with columns means
    #Makes everything numaric and fills in null values
    remaining_na_columns = new_flights_df.loc[:, new_flights_df.isna().sum() > 0].columns
    for column in remaining_na_columns:
        new_flights_df[column] = new_flights_df[column].fillna(new_flights_df[column].mean())
        
    #training the data in X and y
    #I CHANGED IT TO MINUTES_DELAYED, HE HAD IT AS THE 'CANCELLED' COLUMN WITH WE REMOVED
    #Maybe thats why it wont show anything??

    y = new_flights_df['MINUTES_DELAY'].copy()
    X = new_flights_df.drop('MINUTES_DELAY', axis=1).copy()
    
    #Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=123)
    
    #scaling the data with standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns= X.columns)
    
    return X_train, X_test, y_train, y_test

NameError: name 'preprocessing_inputs' is not defined

In [24]:
# Split our preprocessed data into our features and target arrays
y = flights_data_df2["IS_Delayed"].values
X = flights_data_df2.drop(["IS_Delayed"],1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()
model.fit(X_train, y_train)

/Users/sking/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model

LogisticRegression(random_state=1)

In [34]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(random_state=1,n_estimators=100).fit(X_train_scaled, y_train)


In [35]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = eec.predict(X_test_scaled)
EE_Boost_bas = balanced_accuracy_score(y_test, y_pred)
print(EE_Boost_bas)

1.0


In [36]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = eec.predict(X_test_scaled)
confused2 = confusion_matrix(y_test, y_pred)
confused2_df = pd.DataFrame(confused2, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
confused2_df

,Predicted 0,Predicted 1
Actual 0,156020,0
Actual 1,0,106124
